In [80]:
#importation des librairies nécessaire pour le programe
import cv2
import time
import os
import tensorflow as tf
import numpy as np

from tensorflow.python.keras.utils.data_utils import get_file

In [81]:
#mettre le chemin de la vidéo
#ce fichier stabilise une vidéo en prenant un point fixe
#source : eva 
%run ./single_object_tracker.py ./test_final/seq_5.mp4 

4.5.5
out.mp4 file created


In [82]:
#Si érreur : module 'cv2' has no attribute 'legacy', exécuter les lignes suivantes
#pip uninstall opencv-python
#pip uninstall opencv-contrib-python
#pip uninstall opencv-contrib-python-headless
#pip install opencv-contrib-python=4.5.5.62

In [83]:
threshold = 0.5
videoPath = "out.mp4"

In [84]:
#importation de la liste des objets que l'on peut détécter : liste coco
classFile = "coco.names.txt"

with open(classFile, 'r') as f:
    names = f.read().splitlines()

In [85]:
#importation du model entrainé pour détécter les objet. On utilise ici le faster rcnn 
model_url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_800x1333_coco17_gpu-8.tar.gz'

fileName = os.path.basename(model_url)
Name = fileName[:fileName.index('.')]

cacheDir = './pretrained_models'
os.makedirs(cacheDir, exist_ok=True)

get_file(fname=fileName,
        origin=model_url,
        cache_dir=cacheDir,
        cache_subdir="checkpoints",
        extract=True)

model = tf.saved_model.load(os.path.join(cacheDir, "checkpoints", Name, "saved_model"))

In [86]:
#fonction qui permet d'utiliser le modèle chargé précédement sur une image
def createBoundingBox(image):
    colorList = np.random.uniform(low=0, high=255, size=(len(names), 3))
    
    inputTensor = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2RGB)
    inputTensor = tf.convert_to_tensor(inputTensor, dtype=tf.uint8)
    inputTensor = inputTensor[tf.newaxis,...]
    
    detection = model(inputTensor)
    
    bboxs = detection['detection_boxes'][0].numpy()
    Indexes = detection['detection_classes'][0].numpy().astype(np.int32)
    Scores = detection['detection_scores'][0].numpy()
    
    imH, imW, imC = image.shape
    
    bboxIdx = tf.image.non_max_suppression(bboxs, 
                                           Scores, 
                                           max_output_size=50,
                                          iou_threshold=threshold,
                                          score_threshold=threshold)
    
    
    if len(bboxIdx) != 0:
        for i in bboxIdx:
            bbox = tuple(bboxs[i].tolist())
            Confidence = round(100*Scores[i])
            Index = Indexes[i]
            
            LabelText = names[Index-1].upper()
            Color = colorList[Index]
            
            
            
            ymin, xmin, ymax, xmax = bbox
            xmin, xmax, ymin, ymax = (xmin*imW, xmax*imW, ymin*imH, ymax*imH)
            xmin, xmax, ymin, ymax = int(xmin), int(xmax), int(ymin), int(ymax)
            
            displayText = '{}: {}%'.format(LabelText, Confidence)
            
            cv2.rectangle(image, (xmin, ymin), (xmax,ymax), color=Color, thickness=1)
            cv2.putText(image, displayText, (xmin, ymin-10), cv2.FONT_HERSHEY_PLAIN, 1, Color, 2)
            
            #lineWidth = min(int((xmax-xmin)*0.2), int((ymax-ymin)*0.2))
            
            #cv2.line(image, (xmin,ymin), (xmin + lineWidth, ymin), Color, thickness=5)
            
            print(LabelText)
            print('Taille', xmax-xmin, ymax-ymin)
            #print('Positon : ',  xmin + (xmax-xmin)/2, ymin+(ymin-ymax)/2)
            print()
            
    return image

In [87]:
'''''
#detect objects in a picture

imagePath = "./test/test_stop.jpg"

image = cv2.imread(imagePath) 
bbox_image = createBoundingBox(image)
cv2.imshow("Result", bbox_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
'''''

'\'\'\n#detect objects in a picture\n\nimagePath = "./test/test_stop.jpg"\n\nimage = cv2.imread(imagePath) \nbbox_image = createBoundingBox(image)\ncv2.imshow("Result", bbox_image)\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n'

In [88]:
#detect objects in a video using fonction createBoundingBox
cap = cv2.VideoCapture(videoPath)

if(cap.isOpened() == False):
    print("Error opening file ...")

success, image = cap.read()

startTime = 0

while success:
    currentTime = time.time()
    
    fps = 1/(currentTime - startTime)
    startTime = currentTime  
    
    bbox_Image = createBoundingBox(image)
    
    cv2.putText(bbox_Image, "FPS: " + str(int(fps)), (20,70), cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0), 2)
    cv2.imshow("Result", bbox_Image)
    
    key = cv2.waitKey(5) & 0xFF
    if key == ord("e"):
        break
    
    success, image = cap.read()

cv2.destroyAllWindows()

CAR
Taille 71 44

TRAIN
Taille 281 193

TRAIN
Taille 272 265

POTTED PLANT
Taille 59 99

POTTED PLANT
Taille 58 98

POTTED PLANT
Taille 59 100

POTTED PLANT
Taille 59 97

POTTED PLANT
Taille 62 99

POTTED PLANT
Taille 60 100

POTTED PLANT
Taille 63 95

UMBRELLA
Taille 97 39

POTTED PLANT
Taille 64 96

BOTTLE
Taille 23 43

POTTED PLANT
Taille 62 101

BIRD
Taille 176 169

POTTED PLANT
Taille 39 80

POTTED PLANT
Taille 40 76

BIRD
Taille 175 183

AIRPLANE
Taille 85 76

PERSON
Taille 15 34

PERSON
Taille 14 34

PERSON
Taille 17 35

PERSON
Taille 15 36

PERSON
Taille 15 34

PERSON
Taille 17 36



In [89]:
#Rentrer le nom et les valeurs Taille et Position d'un objet proche de l'eau
#pour seq_5 personne à la fin de la vidéo avec une taille de 13,36
Name = "person"
Taille = [17, 36]

In [90]:
#utilisation de ce programme pour extraire une image du programe
%run ./OF-segmentation.py out.mp4 5

Starting segmentation
FPS: 30.0
END


In [91]:
'''
#seq_5 personne: 500; 122

ArrayBlanc = np.load('ArrayBlanc.npy')

coefx = 640*20/100
centerx = 279
xmin, xmax = centerx-coefx, centerx+coefx

coefy = 360*20/100
centery = 214
ymin, ymax = centery-coefy, centery+coefy


cleanArray = []
for points in ArrayBlanc:
    x, y, z = points
    if x>xmin and x<xmax:
        if y>ymin and y<ymax:
            cleanArray.append([x,y,z])
cleanArray
ArrayBlanc.shape
'''

"\n#seq_5 personne: 500; 122\n\nArrayBlanc = np.load('ArrayBlanc.npy')\n\ncoefx = 640*20/100\ncenterx = 279\nxmin, xmax = centerx-coefx, centerx+coefx\n\ncoefy = 360*20/100\ncentery = 214\nymin, ymax = centery-coefy, centery+coefy\n\n\ncleanArray = []\nfor points in ArrayBlanc:\n    x, y, z = points\n    if x>xmin and x<xmax:\n        if y>ymin and y<ymax:\n            cleanArray.append([x,y,z])\ncleanArray\nArrayBlanc.shape\n"

In [92]:
#choisir quatre points dans l'eau, au plus proche de l'objet et appuyer sur 'e'
#extraction du périmètre où l'on va regarder les vecteurs de vitesse
%run ./position.py ./out_frame.png

322   220
375   220
376   237
321   238


In [93]:
xArray = []
yArray = []

with open("./coordonnees_eau.txt") as f:
    for line in f:
        x1, y1 = line.split(' ')
        xArray.append(int(x1))
        yArray.append(int(y1.split('\n')[0]))
        
xmin, xmax = min(xArray), max(xArray)
ymin, ymax = min(yArray), max(yArray)

waterArray = []

y = ymin
while y < ymax:
    i = xmin
    while i < xmax:
        waterArray.append([y,i])
        i += 1
    y +=1

In [94]:
def draw_flow(img, flow, step=16):

    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T
    lines = np.vstack([x, y, x-fx, y-fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)

    img_bgr = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.polylines(img_bgr, lines, 0, (0, 255, 0))

    for (x1, y1), (_x2, _y2) in lines:
        cv2.circle(img_bgr, (x1, y1), 1, (0, 255, 0), -1)

    return img_bgr

In [95]:
#def draw_hsv(flow):

#    h, w = flow.shape[:2]
#    fx, fy = flow[:,:,0], flow[:,:,1]
#    ang = np.arctan2(fy, fx) + np.pi
#    v = np.sqrt(fx*fx+fy*fy)

#    hsv = np.zeros((h, w, 3), np.uint8)
#    hsv[...,0] = ang*(180/np.pi/2)
#    hsv[...,1] = 255
#    hsv[...,2] = np.minimum(v*4, 255)
#    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

#    return None


In [96]:
#trouve des vecteurs de vitesse

cap = cv2.VideoCapture(videoPath)

suc, prev = cap.read()
prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
suc, img = cap.read()

cleanfx, cleanfy = [], []
cfx, cfy = [], []

while suc:

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # start time to calculate FPS
    start = time.time()


    flow = cv2.calcOpticalFlowFarneback(prevgray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    fx, fy = flow[:,:,0], flow[:,:,1]
    for i in range(len(waterArray)):
        cleanfx.append(fx[waterArray[i][0], waterArray[i][1]])
        cleanfy.append(fy[waterArray[i][0], waterArray[i][1]])
    #flow = cv2.calcOpticalFlowFarneback(prevgray, gray, flow=None, pyr_scale=0.5, levels=3, winsize=20, iterations=15, poly_n=5, poly_sigma=1.2, flags=cv2.OPTFLOW_FARNEBACK_GAUSSIAN)
    
    cfx.append(np.mean(np.abs(cleanfx)))
    cfy.append(np.mean(np.abs(cleanfy)))
    
    prevgray = gray


    # End time
    end = time.time()
    # calculate the FPS for current frame detection
    fps = 1 / (end-start)

    #print(f"{fps:.2f} FPS")
    
    img_bgr = draw_flow(gray, flow)
    
    cv2.imshow('flow', img_bgr)
    #cv2.imshow('flow HSV', draw_hsv(flow))
    
    key = cv2.waitKey(5)
    if key == ord('e'):
        break
    
    suc, img = cap.read()
    

cap.release()
cv2.destroyAllWindows()

vitx = np.mean(cfx)
vity = np.mean(cfy)

In [97]:
import cv2 
import datetime 
  
data = cv2.VideoCapture(videoPath) 
  
frames = data.get(cv2.CAP_PROP_FRAME_COUNT) 
fps = int(data.get(cv2.CAP_PROP_FPS)) 
  
seconds = int(frames / fps) 
video_time = str(datetime.timedelta(seconds=seconds)) 
print("duration in seconds:", seconds) 
print("video time:", video_time) 
print(fps)

duration in seconds: 9
video time: 0:00:09
30


In [98]:
#calcul de la vitesse

with open("./value_coco.txt") as f:
    for line in f:
        if Name in line:
            n, x, y, z = line.split(' ')
            break

x, y = int(x), int(y)

ex = x/Taille[0]
ey = y/Taille[0]

vx = vitx*ex
vy = vity*ey

vitesse_eau = np.sqrt(vx**2 + vy**2) * 0.01 * fps
print("la vitesse de l'eau sur cette video est ", vitesse_eau, 'm/s')
vitesse_eau *= 3.6
print("la vitesse de l'eau sur cette video est ", vitesse_eau, 'km/h')

la vitesse de l'eau sur cette video est  3.104975682879396 m/s
la vitesse de l'eau sur cette video est  11.177912458365826 km/h
